In [9]:


import tkinter
from tkinter import *
root=tkinter.Tk()
root.title("Client")
username=" "

topFrame=tkinter.Frame(root,relief=SUNKEN)
lblname=tkinter.Label(topFrame, text="Name: ").pack(side=tkinter.LEFT)
entName=tkinter.Entry(topFrame)
entName.pack(side=tkinter.LEFT)
btnConnect=tkinter.Button(topFrame, text="Connect",command=lambda: connect())
btnConnect.pack(side=tkinter.LEFT)
topFrame.pack(anchor=NW,pady=(5,0))

clientFrame = tkinter.Frame(root,bg="grey",highlightbackground="black",borderwidth=5,relief=SUNKEN)
lblLine = tkinter.Label(clientFrame, text="CLIENT LIST")
lblLine.pack(side=tkinter.TOP)
scrollBar = tkinter.Scrollbar(clientFrame)
scrollBar.pack(side=tkinter.RIGHT, fill=tkinter.Y)
TkDisplay = tkinter.Text(clientFrame,height=100,width=20)
TkDisplay.pack(side=tkinter.LEFT, fill=tkinter.Y, padx=(5,0))
scrollBar.config(command=TkDisplay.yview)
TkDisplay.config(yscrollcommand=scrollBar.set,background="white",highlightbackground="black",state="disabled")
clientFrame.pack(side=LEFT,pady=(5,0))

displayFrame=tkinter.Frame(root,bg="grey",highlightbackground="black",borderwidth=5,relief=SUNKEN)
lblLine=tkinter.Label(displayFrame, text="PUBLIC CHAT").pack()
scrollbar=tkinter.Scrollbar(displayFrame)
scrollbar.pack(side=tkinter.RIGHT, fill=tkinter.Y)
tkDisplay=tkinter.Text(displayFrame,height=20,width=35)
tkDisplay.pack(side=tkinter.LEFT,fill=tkinter.Y,padx=(5,0))
tkDisplay.tag_config("Enter your message",foreground="blue")
scrollbar.config(command=tkDisplay.yview)
tkDisplay.config(yscrollcommand=scrollbar.set, background="white",highlightbackground="white",state="disabled")
displayFrame.pack(anchor=NW,pady=(5,0))


bottomFrame=tkinter.Frame(root,highlightbackground="black",borderwidth=5,relief=SUNKEN)
tkMessage=tkinter.Text(bottomFrame,height=3,width=35)
tkMessage.pack(side=tkinter.LEFT,padx=(5,13),pady=(5,10))
tkMessage.config(highlightbackground="black",state="disabled")
tkMessage.bind("<Return>",(lambda event:getChatMessage(tkMessage.get("1.0",tkinter.END))))
bottomFrame.pack(anchor=NW,pady=(2,0))

import socket

from tkinter import messagebox

import threading
def connect():
    global username
    if len(entName.get())<1:
        tkinter.messagebox.showerror(title="ERRoR!!!",message="You must enter your first name")
    else:
        username=entName.get()
        connect_to_server(username)
        
    


def connect_to_server(name):
   
    global client, HOST_ADDR, HOST_PORT
    client=None
    HOST_ADDR="127.0.0.1"
    HOST_PORT=4920
    
    client=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
        
    client.connect((HOST_ADDR,HOST_PORT))
   
    client.send(bytes(name,"utf8"))
    entName.config(state=tkinter.DISABLED)
    btnConnect.config(state=tkinter.DISABLED) 
    tkMessage.config(state=tkinter.NORMAL)
        
        
    threading._start_new_thread(receive_message_from_server,(client,"m"))
   
        
def receive_message_from_server(sck,m):
    global from_server
    while True:
        from_server=sck.recv(4096)
        if not from_server:
            break
        text_chat=tkDisplay.get("1.0",tkinter.END).strip()
        text_user=tkDisplay.get("1.0",tkinter.END).strip()
        tkDisplay.config(state=tkinter.NORMAL)
        TkDisplay.config(state=tkinter.NORMAL)
        if len(text_user)<1 and len(text_chat)<1:
            print(from_server.decode("utf-8"))
            if "-" in from_server.decode("utf-8") or "exit" in from_server.decode("utf-8") :
                tkDisplay.insert(tkinter.END,from_server.decode("utf-8"))
                tkDisplay.insert(tkinter.END,"\n")
            else:
                TkDisplay.insert(tkinter.END,from_server.decode("utf-8"))
        else:
            if "-" in from_server.decode("utf-8"):
                print(from_server.decode("utf-8"))
                tkDisplay.insert(tkinter.END,from_server.decode("utf-8"))
                # tkDisplay.insert(tkinter.END,"\n")
            else:
                TkDisplay.insert(tkinter.END,from_server.decode("utf-8"))
                            
        tkDisplay.config(state=tkinter.DISABLED)
        tkDisplay.see(tkinter.END)
        TkDisplay.config(state=tkinter.DISABLED)
        TkDisplay.see(tkinter.END)
    sck.close()
    root.destroy()
    
    
    
def getChatMessage(msg):
    msg=msg.replace('\n', '')
    texts=tkDisplay.get("1.0", tkinter.END).strip()
    
    tkDisplay.config(state=tkinter.NORMAL)
    if len(texts)<1:
        tkDisplay.insert(tkinter.END,"You->"+ msg+"\n","tag_your_message")
    else:
        tkDisplay.insert(tkinter.END, "You->"+msg+"\n","tag_your_message")
        
    tkDisplay.config(state=tkinter.DISABLED)
    send_message_to_server(msg)

    tkDisplay.see(tkinter.END)
    tkMessage.delete('1.0',tkinter.END) 
    
def send_message_to_server(msg):
    client.send(msg.encode())
    if msg=="exit":
        client.close()
        root.destroy()
    print("sending message")
            


root.mainloop()